In [1]:
!pip install -Uq apache_beam mwparserfromhell

In [2]:
from itertools import chain

import datasets
from datasets import load_dataset, load_metric

import transformers
import transformers.adapters.composition as ac
from transformers import (
    AdapterTrainer,
    AutoConfig,
    AutoModelForMaskedLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    MultiLingAdapterArguments,
    TrainingArguments,
    set_seed,
)
from transformers.adapters.configuration import AdapterConfig
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
import math

import gc
import sys

NOTE: Redirects are currently not supported in Windows or MacOs.


In [3]:
set_seed(42069)

In [4]:
valid_percentage = 20
language_code = "am"
task_name = language_code + "_mlm"
date = "20221120"

training_args = TrainingArguments(language_code + '_output',
    learning_rate=1e-4,
    num_train_epochs=10.0, report_to="all", 
    gradient_accumulation_steps=4,
    per_device_train_batch_size=2, per_device_eval_batch_size=4)

In [5]:
raw_datasets = load_dataset(
    "wikipedia", language=language_code, date=date, beam_runner='DirectRunner'
)
raw_datasets["validation"] = load_dataset(
    "wikipedia", language=language_code, date=date, 
    split=f"train[:{valid_percentage}%]", beam_runner='DirectRunner'
)
raw_datasets["train"] = load_dataset(
    "wikipedia", language=language_code, date=date, 
    split=f"train[{valid_percentage}%:]", beam_runner='DirectRunner'
)
len(raw_datasets["validation"]), len(raw_datasets["train"]), len(raw_datasets["validation"]) + len(raw_datasets["train"])

Using custom data configuration 20221120.am-date=20221120,language=am
Found cached dataset wikipedia (C:/Users/Thomas/.cache/huggingface/datasets/wikipedia/20221120.am-date=20221120,language=am/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration 20221120.am-date=20221120,language=am
Found cached dataset wikipedia (C:/Users/Thomas/.cache/huggingface/datasets/wikipedia/20221120.am-date=20221120,language=am/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)
Using custom data configuration 20221120.am-date=20221120,language=am
Found cached dataset wikipedia (C:/Users/Thomas/.cache/huggingface/datasets/wikipedia/20221120.am-date=20221120,language=am/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


(2740, 10960, 13700)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")
# model.resize_token_embeddings(len(tokenizer))

In [7]:
adapter_config = AdapterConfig.load("pfeiffer") # AdapterConfig.load("pfeiffer+inv")
model.add_adapter(task_name, config=adapter_config)

In [8]:
model.train_adapter([task_name])
model.set_active_adapters(task_name)

In [9]:
column_names = raw_datasets["train"].column_names
text_column_name = "text"
max_seq_length = tokenizer.model_max_length

In [10]:
def tokenize_function(examples):
    return tokenizer(examples[text_column_name], return_special_tokens_mask=True)

tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names
)

Loading cached processed dataset at C:/Users/Thomas/.cache/huggingface/datasets/wikipedia/20221120.am-date=20221120,language=am/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559\cache-c997e72a580614a1.arrow
Loading cached processed dataset at C:/Users/Thomas/.cache/huggingface/datasets/wikipedia/20221120.am-date=20221120,language=am/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559\cache-15ca0dcbcc2c756a.arrow


In [11]:
def group_texts(examples):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    if total_length >= max_seq_length:
        total_length = (total_length // max_seq_length) * max_seq_length
    result = {
        k: [t[i : i + max_seq_length] for i in range(0, total_length, max_seq_length)]
        for k, t in concatenated_examples.items()
    }
    return result

tokenized_datasets = tokenized_datasets.map(
    group_texts,
    batched=True
)

Loading cached processed dataset at C:/Users/Thomas/.cache/huggingface/datasets/wikipedia/20221120.am-date=20221120,language=am/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559\cache-83306ecabf824b2c.arrow
Loading cached processed dataset at C:/Users/Thomas/.cache/huggingface/datasets/wikipedia/20221120.am-date=20221120,language=am/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559\cache-191f54503cc493bf.arrow


In [12]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]
def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        logits = logits[0]
    return logits.argmax(dim=-1)

metric = load_metric("accuracy")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    labels = labels.reshape(-1)
    preds = preds.reshape(-1)
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return metric.compute(predictions=preds, references=labels)

C:\Users\Thomas\AppData\Local\Temp\ipykernel_18464\2872494391.py:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [13]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm_probability=0.15)

In [14]:
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
)


In [15]:
train_result = trainer.train()
trainer.save_model()

metrics = train_result.metrics
metrics["train_samples"] = len(train_dataset)

trainer.save_metrics("train", metrics)
trainer.save_state()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `XLMRobertaForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\Thomas\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5151
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 6440


Step,Training Loss
500,2.197700
1000,1.978700
1500,1.943200
2000,1.923800
2500,1.903600
3000,1.878300
3500,1.870600
4000,1.866600
4500,1.843500
5000,1.843200


Saving model checkpoint to am_output\checkpoint-500
Configuration saved in am_output\checkpoint-500\am_mlm\adapter_config.json
Module weights saved in am_output\checkpoint-500\am_mlm\pytorch_adapter.bin
Configuration saved in am_output\checkpoint-500\am_mlm\head_config.json
Module weights saved in am_output\checkpoint-500\am_mlm\pytorch_model_head.bin
tokenizer config file saved in am_output\checkpoint-500\tokenizer_config.json
Special tokens file saved in am_output\checkpoint-500\special_tokens_map.json
Saving model checkpoint to am_output\checkpoint-1000
Configuration saved in am_output\checkpoint-1000\am_mlm\adapter_config.json
Module weights saved in am_output\checkpoint-1000\am_mlm\pytorch_adapter.bin
Configuration saved in am_output\checkpoint-1000\am_mlm\head_config.json
Module weights saved in am_output\checkpoint-1000\am_mlm\pytorch_model_head.bin
tokenizer config file saved in am_output\checkpoint-1000\tokenizer_config.json
Special tokens file saved in am_output\checkpoint-10

In [16]:
metrics = trainer.evaluate()

metrics["eval_samples"] = len(eval_dataset)

try:
    perplexity = math.exp(metrics["eval_loss"])
except OverflowError:
    perplexity = float("inf")
    
metrics["perplexity"] = perplexity

trainer.save_metrics("eval", metrics)

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `XLMRobertaForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2696
  Batch size = 4


In [17]:
model.save_adapter(language_code + f'_output/lm_{language_code}_{perplexity:.3f}', task_name)

adapter_loc = f'{language_code}_output/lm_{language_code}_{perplexity:.3f}'

Configuration saved in am_output/lm_am_6.423\adapter_config.json
Module weights saved in am_output/lm_am_6.423\pytorch_adapter.bin
Configuration saved in am_output/lm_am_6.423\head_config.json
Module weights saved in am_output/lm_am_6.423\pytorch_model_head.bin


In [18]:
this = sys.modules[__name__]
for n in dir():
    if n[0]!='_' and n not in ['this', 'adapter_loc', 'gc']: delattr(this, n)
del n
del this
gc.collect()

5587

In [19]:
import pandas as pd
from transformers import AutoAdapterModel, AdapterConfig, AutoTokenizer, AutoConfig
import torch
import numpy as np
from sklearn.metrics import f1_score, balanced_accuracy_score
from transformers.adapters.composition import Stack

In [21]:

en_train = pd.read_csv("data/en_all.csv")
en_train, en_test = np.split(en_train.sample(frac=1, random_state=42), [int(.8*len(en_train))])

am_train = pd.read_csv('data/am/am_train_translated.csv')
am_train, am_dev, am_test = np.split(
    am_train.sample(frac=1, random_state=42), [int(.7*len(am_train)), int(.8*len(am_train))])

In [22]:
combined_train = pd.concat([
    en_train[['text', 'labels']]
])

combined_test = pd.concat([
    am_test[['eng_translated', 'label']].rename(columns={'eng_translated':'text', 'label':'labels'}),
    am_dev[['eng_translated', 'label']].rename(columns={'eng_translated':'text', 'label':'labels'}),
    en_test[['text', 'labels']]
])
test_split_lengths = [('am_test', len(am_test)), ('am_dev', len(am_dev)), ('en_test', len(en_test))]

In [23]:

label2id = {"positive":0, "neutral":1, 'negative':2}
id2label = {0:"positive", 1:"neutral", 2:'negative'}

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def encode_batch(row):
    text = ' '.join(filter(lambda x:x[0]!='@', row.text.split() if type(row.text)==str else []))
    out = tokenizer(text, max_length=100, truncation=True, padding="max_length", return_tensors='pt')
    out['labels'] = torch.LongTensor([label2id[row.labels]])[0]
    return out

train = combined_train.apply(encode_batch, axis=1).reset_index()[0]
test = combined_test.apply(encode_batch, axis=1).reset_index()[0]

len(train), len(test)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Thomas/.cache\huggingface\transformers\87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_vers

(10176, 4341)

In [24]:
config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
)
model = AutoAdapterModel.from_pretrained(
    "xlm-roberta-base",
    config=config,
)

loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Thomas/.cache\huggingface\transformers\87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loadin

In [25]:
lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)
en = model.load_adapter("en/wiki@ukp", config=lang_adapter_config)
task_adapter = model.load_adapter(adapter_loc, config=lang_adapter_config)

model.add_adapter("sa")
model.train_adapter("sa")

model.add_classification_head("sa", num_labels=3)
model.set_active_adapters("sa")

model.active_adapters = Stack(en, "sa")

Attempting to load adapter from source 'ah'...
Found matching adapter at: adapters/ukp/xlm-roberta-base-en-wiki_pfeiffer.json
Resolved adapter files at https://public.ukp.informatik.tu-darmstadt.de/AdapterHub/text_lang/en/xlm-roberta-base/pfeiffer/en_relu_2.zip.
Loading module configuration from ~/.cache\torch\adapters\3e9c8c800af886cd1c684a3acf4929990b35f3ac75e050e942de2ff2ecf895fe-caabba7772816577124631d5b84810c4d32f49afcea4fca637162ec7f432174f-extracted\adapter_config.json
Adding adapter 'en'.
Loading module weights from ~/.cache\torch\adapters\3e9c8c800af886cd1c684a3acf4929990b35f3ac75e050e942de2ff2ecf895fe-caabba7772816577124631d5b84810c4d32f49afcea4fca637162ec7f432174f-extracted\pytorch_adapter.bin
No matching prediction head found in '~/.cache\torch\adapters\3e9c8c800af886cd1c684a3acf4929990b35f3ac75e050e942de2ff2ecf895fe-caabba7772816577124631d5b84810c4d32f49afcea4fca637162ec7f432174f-extracted'
Loading module configuration from am_output/lm_am_6.423\adapter_config.json
Adding 

In [26]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=7,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True
#     # The next line is important to ensure the dataset labels are properly passed to the model
#     remove_unused_columns=False,
)

def compute_scores(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    i, output = 0, dict()
    for name, split_length in test_split_lengths:
        s = np.s_[i:i+split_length]
        split_preds = preds[s]
        split_labels = p.label_ids[s]
        output[f'{name}_acc'] = (split_preds==split_labels).mean()
        output[f'{name}_weighted_f1'] = f1_score(split_preds, split_labels, average='weighted')
        output[f'{name}_balanced_accurancy'] = balanced_accuracy_score(split_preds, split_labels)
        i += split_length
    return output

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_scores,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [27]:
trainer.train()

C:\Users\Thomas\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10176
  Num Epochs = 7
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2226


Step,Training Loss
200,0.872800
400,0.637900
600,0.566500
800,0.549900
1000,0.525500
1200,0.509800
1400,0.473900
1600,0.489000
1800,0.469100
2000,0.462000


Saving model checkpoint to ./training_output\checkpoint-500
Configuration saved in ./training_output\checkpoint-500\en\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\en\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\am_mlm\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\am_mlm\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\am_mlm\head_config.json
Module weights saved in ./training_output\checkpoint-500\am_mlm\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\sa\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\sa\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\sa\head_config.json
Module weights saved in ./training_output\checkpoint-500\sa\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\am_mlm\head_config.json
Module weights saved in ./training_output\checkpo

TrainOutput(global_step=2226, training_loss=0.545309661426527, metrics={'train_runtime': 500.1931, 'train_samples_per_second': 142.409, 'train_steps_per_second': 4.45, 'total_flos': 4114009274457600.0, 'train_loss': 0.545309661426527, 'epoch': 7.0})

In [28]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 4341
  Batch size = 32


{'eval_loss': 0.9375244379043579,
 'eval_am_test_acc': 0.48872180451127817,
 'eval_am_test_weighted_f1': 0.4814607164216763,
 'eval_am_test_balanced_accurancy': 0.4749860692959444,
 'eval_am_dev_acc': 0.5008347245409015,
 'eval_am_dev_weighted_f1': 0.4979691973669414,
 'eval_am_dev_balanced_accurancy': 0.4836828942903495,
 'eval_en_test_acc': 0.7728880157170923,
 'eval_en_test_weighted_f1': 0.7723368193497595,
 'eval_en_test_balanced_accurancy': 0.7552755931073509,
 'eval_runtime': 14.5884,
 'eval_samples_per_second': 297.564,
 'eval_steps_per_second': 9.322,
 'epoch': 7.0}

In [29]:
del trainer
gc.collect()

274

In [30]:
model.active_adapters = Stack(task_adapter, "sa")

In [32]:
combined_test = pd.concat([
    am_test[['tweet', 'label']].rename(columns={'tweet':'text', 'label':'labels'}),
    am_dev[['tweet', 'label']].rename(columns={'tweet':'text', 'label':'labels'}),
#     en_test[['translated', 'labels']].rename(columns={'translated':'text'})
])
test = combined_test.apply(encode_batch, axis=1).reset_index()[0]

In [33]:
eval_trainer = AdapterTrainer(
    model=model,
    args=TrainingArguments(output_dir="./eval_output", remove_unused_columns=False,),
    eval_dataset=test,
    compute_metrics=compute_scores
)
eval_trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Evaluation *****
  Num examples = 1796
  Batch size = 8


C:\Users\Thomas\AppData\Local\Temp\ipykernel_18464\3147666310.py:23: RuntimeWarning: Mean of empty slice.
  output[f'{name}_acc'] = (split_preds==split_labels).mean()
C:\Users\Thomas\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Thomas\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Thomas\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'eval_loss': 1.2036296129226685,
 'eval_am_test_acc': 0.5764411027568922,
 'eval_am_test_weighted_f1': 0.5882240998528119,
 'eval_am_test_balanced_accurancy': 0.544426904813819,
 'eval_am_dev_acc': 0.5575959933222037,
 'eval_am_dev_weighted_f1': 0.5692377152677427,
 'eval_am_dev_balanced_accurancy': 0.5388372830536815,
 'eval_en_test_acc': nan,
 'eval_en_test_weighted_f1': 0.0,
 'eval_en_test_balanced_accurancy': nan,
 'eval_runtime': 6.651,
 'eval_samples_per_second': 270.033,
 'eval_steps_per_second': 33.829}

In [34]:
config = AutoConfig.from_pretrained(
    "xlm-roberta-base",
)
model = AutoAdapterModel.from_pretrained(
    "xlm-roberta-base",
    config=config,
)

loading configuration file https://huggingface.co/xlm-roberta-base/resolve/main/config.json from cache at C:\Users\Thomas/.cache\huggingface\transformers\87683eb92ea383b0475fecf99970e950a03c9ff5e51648d6eee56fb754612465.dfaaaedc7c1c475302398f09706cbb21e23951b73c6e2b3162c1c8a99bb3b62a
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loadin

In [35]:
lang_adapter_config = AdapterConfig.load("pfeiffer", reduction_factor=2)
# en = model.load_adapter("en/wiki@ukp", config=lang_adapter_config)
task_adapter = model.load_adapter(adapter_loc, config=lang_adapter_config)

model.add_adapter("sa")
model.train_adapter("sa")

model.add_classification_head("sa", num_labels=3)
model.set_active_adapters("sa")

model.active_adapters = Stack(task_adapter, "sa")

Loading module configuration from am_output/lm_am_6.423\adapter_config.json
Adding adapter 'am_mlm'.
Loading module weights from am_output/lm_am_6.423\pytorch_adapter.bin
Loading module configuration from am_output/lm_am_6.423\head_config.json
Adding head 'am_mlm' with config {'head_type': 'masked_lm', 'vocab_size': 250002, 'layers': 2, 'activation_function': 'gelu', 'layer_norm': True, 'bias': True, 'shift_labels': False, 'label2id': None}.
Loading module weights from am_output/lm_am_6.423\pytorch_model_head.bin
Some weights of the state_dict could not be loaded into model: lm_head.bias
Adding adapter 'sa'.
Adding head 'sa' with config {'head_type': 'classification', 'num_labels': 3, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 2}, 'use_pooler': False, 'bias': True}.


In [36]:
combined_train = pd.concat([
    am_train[['tweet', 'label']].rename(columns={'tweet':'text', 'label':'labels'})
#     en_train[['translated', 'labels']].rename(columns={'translated':'text'})
])

train = combined_train.apply(encode_batch, axis=1).reset_index()[0]

In [37]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=7,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True
)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
    compute_metrics=compute_scores,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [38]:

trainer.train()

C:\Users\Thomas\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4188
  Num Epochs = 7
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 917


Step,Training Loss
200,1.014600
400,0.912100
600,0.878300
800,0.867000


Saving model checkpoint to ./training_output\checkpoint-500
Configuration saved in ./training_output\checkpoint-500\am_mlm\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\am_mlm\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\am_mlm\head_config.json
Module weights saved in ./training_output\checkpoint-500\am_mlm\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\sa\adapter_config.json
Module weights saved in ./training_output\checkpoint-500\sa\pytorch_adapter.bin
Configuration saved in ./training_output\checkpoint-500\sa\head_config.json
Module weights saved in ./training_output\checkpoint-500\sa\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\am_mlm\head_config.json
Module weights saved in ./training_output\checkpoint-500\am_mlm\pytorch_model_head.bin
Configuration saved in ./training_output\checkpoint-500\sa\head_config.json
Module weights saved in ./training_output\che

TrainOutput(global_step=917, training_loss=0.9089419116370681, metrics={'train_runtime': 191.7851, 'train_samples_per_second': 152.859, 'train_steps_per_second': 4.781, 'total_flos': 1563199660159200.0, 'train_loss': 0.9089419116370681, 'epoch': 7.0})

In [39]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1796
  Batch size = 32


C:\Users\Thomas\AppData\Local\Temp\ipykernel_18464\3147666310.py:23: RuntimeWarning: Mean of empty slice.
  output[f'{name}_acc'] = (split_preds==split_labels).mean()
C:\Users\Thomas\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Thomas\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Thomas\anaconda3\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'eval_loss': 0.8390172719955444,
 'eval_am_test_acc': 0.631578947368421,
 'eval_am_test_weighted_f1': 0.6565461977236483,
 'eval_am_test_balanced_accurancy': 0.6081060606060606,
 'eval_am_dev_acc': 0.6076794657762938,
 'eval_am_dev_weighted_f1': 0.6293359375035844,
 'eval_am_dev_balanced_accurancy': 0.6080657640112702,
 'eval_en_test_acc': nan,
 'eval_en_test_weighted_f1': 0.0,
 'eval_en_test_balanced_accurancy': nan,
 'eval_runtime': 5.7778,
 'eval_samples_per_second': 310.847,
 'eval_steps_per_second': 9.865,
 'epoch': 7.0}